In [ ]:
import sys
!{sys.executable} -m pip install customtkinter 
import customtkinter as ctk
import sqlite3
from tkinter import messagebox

conn =sqlite3.connect('fitness_tracker.db')
cursor =conn.cursor()


def save_data():
    exercise =exercise_entry.get()
    duration=duration_entry.get()
    ex_type =type_entry.get()
    completed =1 if completed_var.get() else 0
    inten =intensity.get()
    date_val=date_entry.get()

    if not exercise or not duration or not date_val:
        messagebox.showwarning("Missing Data","Please fill all requires fields.")
        return
    cursor.execute("""
    INSERT INTO exercises(exercise,duration,type,completed,intensity,date)
    VALUES(?,?,?,?,?,?)
    """,(exercise,duration,ex_type,completed,inten,date_val))
    conn.commit()
    messagebox.showinfo("Saved",f"Exercise '{exercise}' recorded!")


def view_entries():
    view_win=ctk.CTkToplevel(tracker)
    view_win.title("Saved Exercises")
    view_win.geometry("500x400")

    cursor.execute("""
    SELECT exercise,duration,type,completed,intensity,date
    FROM exercises
    ORDER BY date DESC
    """)
    rows =cursor.fetchall()
    
    scrollable_frame=ctk.CTkScrollableFrame(view_win,width=380,height=350)
    scrollable_frame.pack(padx=10,pady=10,fill="both",expand=True)

  
    if not rows:
        ctk.CTkLabel(scrollable_frame,text="No records found.")
    else:
        for r in rows:
            exercise,duration,ex_type,completed,inten,date_val= r
            line =f"{date_val}|{exercise}|{duration} mins|{ex_type}|{'done' if completed else 'not done'}|{inten}\n"
            ctk.CTkLabel(scrollable_frame,text=line,anchor="w").pack(fill="x",padx=5,pady=2)
     
   
def on_close():
    conn.close()
    tracker.destroy()


#---------------GUI------------------------


tracker=ctk.CTk()
tracker.geometry("800x600")
tracker.title("Fitness tracker")

lbl_exercise =ctk.CTkLabel(tracker,text="Exercise Name")
lbl_exercise.grid(row=0,column=0,padx=2,pady=3,sticky="e")
exercise_entry=ctk.CTkEntry(tracker,placeholder_text="eg.Pushups")
exercise_entry.grid(row=0,column=1,padx=2,pady=3)

lbl_duration=ctk.CTkLabel(tracker,text="Duration(minutes)")
lbl_duration.grid(row=1,column=0,padx=10,pady=10,sticky="e")
duration_entry=ctk.CTkEntry(tracker,placeholder_text="eg.30")
duration_entry.grid(row=1,column=1,padx=20,pady=10)

lbl_type=ctk.CTkLabel(tracker,text="Exercise Type")
lbl_type.grid(row=2,column=0,padx=10,pady=10,sticky="e")
type_entry=ctk.StringVar(value="Strength")
type_entry=ctk.CTkOptionMenu(tracker,values=["Cardio","Strength","Yoga","Other"],variable=type_entry)
type_entry.grid(row=2,column=1,padx=20,pady=10)

completed_var=ctk.BooleanVar()
completed_check=ctk.CTkCheckBox(tracker,text="Completed?",variable=completed_var)
completed_check.grid(row=8,column=1,padx=20,pady=20,sticky="w")

lbl_intensity=ctk.CTkLabel(tracker,text="Intensity")
lbl_intensity.grid(row=3,column=0,padx=10,pady=10,sticky="e")
intensity=ctk.StringVar(value="Medium")
ctk.CTkRadioButton(tracker,text="Low",variable=intensity,value="Low").grid(row=4,column=1,padx=20,pady=10)
ctk.CTkRadioButton(tracker,text="Medium",variable=intensity,value="Medium").grid(row=5,column=1,padx=20,pady=10)
ctk.CTkRadioButton(tracker,text="High",variable=intensity,value="High").grid(row=6,column=1,padx=20,pady=10)

lbl_date=ctk.CTkLabel(tracker,text="Date")
lbl_date.grid(row =7,column=0,padx=10,pady=10,sticky="e")
date_entry=ctk.CTkEntry(tracker,placeholder_text="YYYY-MM-DD")
date_entry.grid(row=7,column=1,padx=20,pady=10)

btn_save=ctk.CTkButton(tracker,text="Save Exercise",command=save_data)
btn_save.grid(row=9,column=1,columnspan=2,pady=10)

btn_view =ctk.CTkButton(tracker,text="View Entries",command=view_entries)
btn_view.grid(row=10,column=1,columnspan=2,pady=5)

tracker.grid_columnconfigure(0,weight=1)
tracker.grid_columnconfigure(1,weight=0)
tracker.grid_columnconfigure(2,weight=0)
tracker.grid_columnconfigure(3,weight=1)


tracker.protocol("WM_DELETE_WINDOW",on_close)
tracker.mainloop()



